In [1]:
pip install spotipy

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.

   ---------------------------------------- 0.0/252.0 kB ? eta -:--:--
   ------ --------------------------------- 41.0/252.0 kB 991.0 kB/s eta 0:00:01
   --------------------------- ------------ 174.1/252.0 kB 2.1 MB/s eta 0:00:01
   ---------------------------------------- 252.0/252.0 kB 2.2 MB/s eta 0:00:00


In [94]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials as scc
import pandas as pd

In [75]:
# Creating a Spotify obj to authenticate

client_credentials_manager = scc(client_id='c872b1a8eff1460ab561726a9f0cad07', client_secret='1593529db9b14c8ea33405ba945a355d')

In [76]:
# Creating a Spotify obj to be able to use/extract data from Spotify once it has been authorized

sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [77]:
playlist_link = 'https://open.spotify.com/playlist/37i9dQZEVXbLRQDuF5jeBp'

In [78]:
# Extracting the URI from the URL above

playlist_URI = playlist_link.split('/')[-1].split('?')[0]

In [79]:
# Creating dictionary with tracks from the playlist

data = sp.playlist_tracks(playlist_URI)

In [85]:
# We did this after the loop but creating the list of the album/track dictionaries

album_list = []

# loop for album info only

for row in data['items']:
    # album ID = https://open.spotify.com/album/1Mo4aZ8pdj6L1jx8zSwJnt
    album_id = row['track']['album']['id']
    # album name
    album_name = row['track']['album']['name']
    # release date of album
    album_date = row['track']['album']['release_date']
    # total tracks from album
    album_trackNo = row['track']['album']['total_tracks']
    # album spotify URL (similar to album ID)
    album_url = row['track']['album']['external_urls']['spotify']
    # album artist
    # album_artist = row['track']['album']['artists'][0]['name']
    
    # creating a dictionary for the above extracted data
    album_element = {
                        'album_id':album_id,
                         'album_name':album_name,
                         'album_date':album_date,
                         'album_trackNo':album_trackNo,
                         'album_url':album_url
                        # ,
                        # 'album_artist':album_artist
                    }
    # adding the top 50 USA song info into the initialized list
    album_list.append(album_element)

In [89]:
# We did this after the loop but creating the list of the artist dictionaries

artist_list = []

# loop for album artist/s only

for row in data['items']:
    for key, value in row.items():
        if key == 'track':
            for artist in value['artists']:
                artist_dict = {
                                'artist_id':artist['id'],
                                'artist_name':artist['name'],
                                'external_url':artist['href']
                                }
                artist_list.append(artist_dict)

In [92]:
# We did this after the loop but creating the list of the song/track dictionaries

song_list = []

# loop for tracks only

for row in data['items']:
    song_id = row['track']['id']
    song_name = row['track']['name']
    song_duration = row['track']['duration_ms']
    song_url = row['track']['external_urls']['spotify']
    song_popularity = row['track']['popularity']
    song_added = row['added_at']
    album_id = row['track']['album']['id']
    artist_id = row['track']['album']['artists'][0]['id']

    song_element = {
                        'song_id':song_id,
                        'song_name':song_name,
                        'song_duration':song_duration,
                        'song_url':song_url,
                        'song_popularity':song_popularity,
                        'song_added':song_added,
                        'album_id':album_id,
                        'artist_id':artist_id
                    }
    song_list.append(song_element)

In [106]:
# putting extracted album data into dataframe

album_df = pd.DataFrame.from_dict(album_list)

# trimming fat/getting distinct album list

album_df = album_df.drop_duplicates(subset=['album_id'])

In [113]:
# putting extracted artist data into dataframe

artist_df = pd.DataFrame.from_dict(artist_list)

# trimming fat/getting distinct artist list

artist_df = artist_df.drop_duplicates(subset=['artist_id'])

In [120]:
# putting extracted song data into dataframe

song_df = pd.DataFrame.from_dict(song_list)

# trimming fat/getting distinct song list

song_df = song_df.drop_duplicates(subset=['song_id'])

In [125]:
# converting column/s in album_df to be datetime format

album_df['album_date'] = pd.to_datetime(album_df['album_date'])

In [126]:
# converting column/s in song_df to be datetime format

song_df['song_added'] = pd.to_datetime(song_df['song_added'])